In [1]:
!pip install -qU pgmpy pandas


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 60.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 37.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.3 which is incompatible.


In [2]:
from pgmpy.models import DiscreteBayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.sampling import BayesianModelSampling
from pgmpy.estimators import BayesianEstimator
from pgmpy.inference import VariableElimination
import pandas as pd

# ==============================================================
# (a) Network Structure
# ==============================================================

model = DiscreteBayesianNetwork([
    ("TrafficLevel", "AQI"),
    ("IndustrialActivity", "AQI"),
    ("WeatherCondition", "AQI"),
    ("AQI", "RespiratoryCases"),
    ("WeatherCondition", "RespiratoryCases"),
])

# ==============================================================
# (b) True CPDs for Synthetic Data Generation
# ==============================================================

cpd_T = TabularCPD(
    variable="TrafficLevel",
    variable_card=2,
    values=[[0.3], [0.7]],
    state_names={"TrafficLevel": ["low", "high"]}
)

cpd_I = TabularCPD(
    variable="IndustrialActivity",
    variable_card=2,
    values=[[0.4], [0.6]],
    state_names={"IndustrialActivity": ["low", "high"]}
)

cpd_W = TabularCPD(
    variable="WeatherCondition",
    variable_card=2,
    values=[[0.5], [0.5]],
    state_names={"WeatherCondition": ["favourable", "unfavourable"]}
)

cpd_AQI = TabularCPD(
    variable="AQI",
    variable_card=3,
    values=[
        [0.8, 0.6, 0.6, 0.4, 0.5, 0.3, 0.3, 0.1],   # good
        [0.15, 0.25, 0.25, 0.3, 0.3, 0.3, 0.35, 0.3],  # moderate
        [0.05, 0.15, 0.15, 0.3, 0.2, 0.4, 0.35, 0.6],  # poor
    ],
    evidence=["TrafficLevel", "IndustrialActivity", "WeatherCondition"],
    evidence_card=[2, 2, 2],
    state_names={
        "AQI": ["good", "moderate", "poor"],
        "TrafficLevel": ["low", "high"],
        "IndustrialActivity": ["low", "high"],
        "WeatherCondition": ["favourable", "unfavourable"],
    }
)

cpd_R = TabularCPD(
    variable="RespiratoryCases",
    variable_card=2,
    values=[
        [0.9, 0.8, 0.7, 0.5, 0.4, 0.2],  # low
        [0.1, 0.2, 0.3, 0.5, 0.6, 0.8],  # high
    ],
    evidence=["AQI", "WeatherCondition"],
    evidence_card=[3, 2],
    state_names={
        "RespiratoryCases": ["low", "high"],
        "AQI": ["good", "moderate", "poor"],
        "WeatherCondition": ["favourable", "unfavourable"],
    }
)

model.add_cpds(cpd_T, cpd_I, cpd_W, cpd_AQI, cpd_R)
model.check_model()

# ==============================================================
# (c) Generate Synthetic Data (300 samples)
# ==============================================================

sampler = BayesianModelSampling(model)
data = sampler.forward_sample(size=300, seed=42)

learned_model = DiscreteBayesianNetwork(model.edges())
learned_model.fit(
    data,
    estimator=BayesianEstimator,
    prior_type="BDeu"
)

infer = VariableElimination(learned_model)

# ==============================================================
# Helper (new pgmpy DiscreteFactor API)
# ==============================================================

def print_prob(desc, q, var, state):
    idx = q.state_names[var].index(state)
    p = float(q.values[idx])
    print(f"{desc}: {p:.4f}")

# ==============================================================
# (d) Inference Queries
# ==============================================================

# 1) P(AQI = good | TrafficLevel = low)
q1 = infer.query(["AQI"], evidence={"TrafficLevel": "low"})
print_prob("P(AQI=good | T=low)", q1, "AQI", "good")

# 2) P(RespiratoryCases = low | AQI = good)
q2 = infer.query(["RespiratoryCases"], evidence={"AQI": "good"})
print_prob("P(R=low | AQI=good)", q2, "RespiratoryCases", "low")

# 3) Non-trivial: P(AQI = poor | I=high, W=unfavourable)
q3 = infer.query(
    ["AQI"],
    evidence={"IndustrialActivity": "high", "WeatherCondition": "unfavourable"}
)
print_prob("P(AQI=poor | I=high, W=unfavourable)", q3, "AQI", "poor")

# 4) Non-trivial: P(R=low | T=low, I=low)
q4 = infer.query(
    ["RespiratoryCases"],
    evidence={"TrafficLevel": "low", "IndustrialActivity": "low"}
)
print_prob("P(R=low | T=low, I=low)", q4, "RespiratoryCases", "low")

# ==============================================================
# (e) Intervention: do(TrafficLevel = low)
# ==============================================================

# Get learned state names (they must match exactly)
orig_states = learned_model.get_cpds("TrafficLevel").state_names["TrafficLevel"]

# Deterministic CPD: do(T=low)
cpd_T_do_low = TabularCPD(
    variable="TrafficLevel",
    variable_card=2,
    values=[[1.0], [0.0]],  # Always "low"
    state_names={"TrafficLevel": orig_states}
)

# Build intervened model
intervened_model = DiscreteBayesianNetwork(model.edges())
intervened_model.add_cpds(
    cpd_T_do_low,
    learned_model.get_cpds("IndustrialActivity"),
    learned_model.get_cpds("WeatherCondition"),
    learned_model.get_cpds("AQI"),
    learned_model.get_cpds("RespiratoryCases"),
)

intervened_model.check_model()
infer_do = VariableElimination(intervened_model)

# Interventional probability
q_do = infer_do.query(["RespiratoryCases"])
print_prob("P(R=low | do(T=low))", q_do, "RespiratoryCases", "low")

# Observational probability (for comparison)
q_obs = infer.query(["RespiratoryCases"], evidence={"TrafficLevel": "low"})
print_prob("P(R=low | T=low) (observational)", q_obs, "RespiratoryCases", "low")


  0%|          | 0/5 [00:00<?, ?it/s]

P(AQI=good | T=low): 0.4907
P(R=low | AQI=good): 0.8578
P(AQI=poor | I=high, W=unfavourable): 0.5035
P(R=low | T=low, I=low): 0.6667
P(R=low | do(T=low)): 0.5536
P(R=low | T=low) (observational): 0.6544
